In [1]:
import trappy
from trappy import ftrace
from trappy.ftrace import GenericFTrace
import numpy as np
import pandas

In [2]:
lines = [
"            adbd-5709  [007]  2943.184105: sched_contrib_scale_f: cpu=7 cpu_scale_factor=1\n"
"            adbd-5709  [007]  2943.184105: sched_load_avg_cpu:   cpu=7 util_avg=825\n"
"     ->transport-5713  [006]  2943.184106: sched_load_avg_cpu:   cpu=6 util_avg=292\n"
"     ->transport-5713  [006]  2943.184107: sched_contrib_scale_f: cpu=6 cpu_scale_factor=2\n"
"            adbd-5709  [007]  2943.184108: sched_load_avg_cpu:   cpu=7 util_avg=850\n"
"            adbd-5709  [007]  2943.184109: sched_contrib_scale_f: cpu=7 cpu_scale_factor=3\n"
"            adbd-5709  [007]  2943.184110: sched_load_avg_cpu:   cpu=6 util_avg=315\n"
]

In [3]:
with open('/tmp/trappy_mergedf_example.txt', 'w') as fh:
    for line in lines:
        fh.write(line)

In [4]:
!cat /tmp/trappy_mergedf_example.txt

            adbd-5709  [007]  2943.184105: sched_contrib_scale_f: cpu=7 cpu_scale_factor=1
            adbd-5709  [007]  2943.184105: sched_load_avg_cpu:   cpu=7 util_avg=825
     ->transport-5713  [006]  2943.184106: sched_load_avg_cpu:   cpu=6 util_avg=292
     ->transport-5713  [006]  2943.184107: sched_contrib_scale_f: cpu=6 cpu_scale_factor=2
            adbd-5709  [007]  2943.184108: sched_load_avg_cpu:   cpu=7 util_avg=850
            adbd-5709  [007]  2943.184109: sched_contrib_scale_f: cpu=7 cpu_scale_factor=3
            adbd-5709  [007]  2943.184110: sched_load_avg_cpu:   cpu=6 util_avg=315


In [5]:
trace = ftrace.FTrace('/tmp/trappy_mergedf_example.txt', events=['sched_contrib_scale_f', 'sched_load_avg_cpu'], normalize_time=False)

In [6]:
df1 = trace.sched_load_avg_cpu.data_frame[['cpu', 'util_avg', '__line']]
df1

,cpu,util_avg,__line
Time,,,
2943.184105,7,825,1
2943.184106,6,292,2
2943.184108,7,850,4
2943.184110,6,315,6


In [7]:
df2 = trace.sched_contrib_scale_f.data_frame[['cpu', 'cpu_scale_factor', '__line']]
df2

,cpu,cpu_scale_factor,__line
Time,,,
2943.184105,7,1,0
2943.184107,6,2,3
2943.184109,7,3,5


In [8]:
trappy.utils.merge_dfs(df1, df2, 'cpu')

/home/joelaf/repo/lisa-aosp/external/trappy/trappy/utils.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  df = pd.concat([pr_df, sec_df], keys=['primary', 'secondary']).sort(columns='__line')


,__line,cpu,cpu_scale_factor,util_avg
Time,,,,
2943.184105,1,7,1.0,825.0
2943.184106,2,6,NaN,292.0
2943.184108,4,7,1.0,850.0
2943.184110,6,6,2.0,315.0
